In [1]:
%matplotlib inline

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import os
import sys
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm

from ast import literal_eval

""
sns.set_context("notebook", font_scale=1.5)
plt.rcParams['figure.figsize'] = (17, 5)

import platform
if 'Windows' in platform.platform():
    ROOT_PATH = "D:/PycharmProjects/HMAN"
else:
    ROOT_PATH = "/home/xkliu/PycharmProjects/HMAN"
RAW_DATA_PATH = ROOT_PATH  + "/raw_data"
DATA_PATH = ROOT_PATH + "/data"
os.chdir(ROOT_PATH)
sys.path.append("./")

from kddirkit.utils import utils


In [2]:
import logging
import torch.nn as nn
import numpy as np

from kddirkit.networks.encoders import SentenceEncoder
from kddirkit.networks.models import BaselineModel
from kddirkit.config import *
# from kddirkit.dataloaders import LoadNYT, LoadHierData
from kddirkit.frameworks import Trainer
from kddirkit.losses.FocalLoss import FocalLoss

project_name = 'HMAN'

logger = logging.getLogger(project_name)

## 1. Load Data

In [3]:
track_dtype = {'track_id': int, 'album_id': int, 'album_type': str, 'artist_id': int, 'set_split': str,
               'set_subset': str, 'track_genre_top': str, 'track_genres': str, 'track_genres_all': str,
               'track_title': str}
genres_converters = {'track_genres': literal_eval, 'track_genres_all': literal_eval}
medium_data = pd.read_csv(RAW_DATA_PATH + '/medium_data.csv', converters=genres_converters)

In [4]:
medium_data_train = pd.read_csv(RAW_DATA_PATH + '/medium_data_train.csv', converters=genres_converters)
medium_data_test = pd.read_csv(RAW_DATA_PATH + '/medium_data_test.csv', converters=genres_converters)
medium_data_val = pd.read_csv(RAW_DATA_PATH + '/medium_data_val.csv', converters=genres_converters)

In [5]:
medium_instance_triple = np.load(DATA_PATH + '/' + 'medium_instance_triple.npy')
medium_instance_scope = np.load(DATA_PATH + '/' + 'medium_instance_scope.npy')
medium_label = np.load(DATA_PATH + '/' + 'medium_label.npy')

In [6]:
medium_train_instance_triple = np.load(DATA_PATH + '/' + 'medium_train_instance_triple.npy')
medium_train_instance_scope = np.load(DATA_PATH + '/' + 'medium_train_instance_scope.npy')
medium_train_label = np.load(DATA_PATH + '/' + 'medium_train_label.npy')

In [7]:
medium_val_instance_triple = np.load(DATA_PATH + '/' + 'medium_val_instance_triple.npy')
medium_val_instance_scope = np.load(DATA_PATH + '/' + 'medium_val_instance_scope.npy')
medium_val_label = np.load(DATA_PATH + '/' + 'medium_val_label.npy')

In [8]:
medium_test_entity_pair = np.load(DATA_PATH + '/' + 'medium_test_entity_pair.npy')
medium_test_entity_scope = np.load(DATA_PATH + '/' + 'medium_test_entity_scope.npy')
medium_test_label = np.load(DATA_PATH + '/' + 'medium_test_label.npy')

In [9]:
medium_label_transform = np.load(DATA_PATH + '/' + 'medium_label_transform.npy')
medium_train_label_transform = np.load(DATA_PATH + '/' + 'medium_train_label_transform.npy')
medium_val_label_transform = np.load(DATA_PATH + '/' + 'medium_val_label_transform.npy')
medium_test_label_transform = np.load(DATA_PATH + '/' + 'medium_test_label_transform.npy')
medium_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_label_bottom_transform.npy')
medium_train_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_train_label_bottom_transform.npy')
medium_val_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_val_label_bottom_transform.npy')
medium_test_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_test_label_bottom_transform.npy')

In [10]:
col_name = ['track_id', 'album_id', 'album_type', 'artist_id', 'set_split', 'set_subset', 'track_genres_top', 'track_genre', 'track_genres_all']
medium_data_train_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_train_sort.txt', sep ='-----',  skiprows =1, names  = col_name)
medium_data_val_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_VAL_sort.txt', sep = '-----',  skiprows =1, names  = col_name)
medium_data_test_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_test_sort.txt', sep = '-----', skiprows =1, names  = col_name)

C:\Users\Johan\AppData\Local\Temp\ipykernel_38168\691608824.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  medium_data_train_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_train_sort.txt', sep ='-----',  skiprows =1, names  = col_name)
C:\Users\Johan\AppData\Local\Temp\ipykernel_38168\691608824.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  medium_data_val_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_VAL_sort.txt', sep = '-----',  skiprows =1, names  = col_name)
C:\Users\Johan\AppData\Local\Temp\ipykernel_38168\691608824.py:4: ParserWarning: Falling back to the 'python' engine 

In [11]:
medium_data

,track_id,album_id,album_type,artist_id,set_split,set_subset,track_genre_top,track_genres,track_genres_all,track_title
0,2,1,Album,1,training,small,Hip-Hop,[21],[21],Food
1,5,1,Album,1,training,small,Hip-Hop,[21],[21],This World
2,10,6,Album,6,training,small,Pop,[10],[10],Freeway
3,140,61,Album,54,training,small,Folk,[17],[17],Queen Of The Wires
4,141,60,Album,54,training,small,Folk,[17],[17],Ohio
...,...,...,...,...,...,...,...,...,...,...
24995,155297,22935,Album,24354,training,medium,Instrumental,"[18, 107, 1235]","[107, 18, 1235]",Nebula Reborn
24996,155298,22936,Album,22050,training,medium,Folk,"[17, 103]","[17, 103]",An Idiot Abroad
24997,155306,22936,Album,22050,training,medium,Folk,"[17, 103]","[17, 103]",Tiny Man
24998,155307,22937,Live Performance,7820,training,medium,Experimental,[1],"[1, 38]",Kolka


In [12]:
medium_data_train_sort

,track_id,album_id,album_type,artist_id,set_split,set_subset,track_genres_top,track_genre,track_genres_all
0,2,1,Album,1,training,small,Hip-Hop,[21],[21]
1,5,1,Album,1,training,small,Hip-Hop,[21],[21]
2,3,1,Album,1,training,medium,Hip-Hop,[21],[21]
3,134,1,Album,1,training,medium,Hip-Hop,[21],[21]
4,10666,1,Album,1,training,medium,Hip-Hop,[21],[21]
...,...,...,...,...,...,...,...,...,...
19917,155297,22935,Album,24354,training,medium,Instrumental,"[18, 107, 1235]","[107, 18, 1235]"
19918,155298,22936,Album,22050,training,medium,Folk,"[17, 103]","[17, 103]"
19919,155306,22936,Album,22050,training,medium,Folk,"[17, 103]","[17, 103]"
19920,155307,22937,Live Performance,7820,training,medium,Experimental,[1],"[1, 38]"


In [13]:
# Load metadata and features.
tracks = utils.load(RAW_DATA_PATH + '/fma_metadata/tracks.csv')
genres = utils.load(RAW_DATA_PATH + '/fma_metadata/genres.csv')
features = utils.load(RAW_DATA_PATH + '/fma_metadata/features.csv')
echonest = utils.load(RAW_DATA_PATH + '/fma_metadata/echonest.csv')

In [14]:
medium_data_train_sort.track_id

0             2
1             5
2             3
3           134
4         10666
          ...  
19917    155297
19918    155298
19919    155306
19920    155307
19921    155314
Name: track_id, Length: 19922, dtype: int64

In [15]:
small = tracks['set', 'subset'] <= 'medium'

train = tracks['set', 'split'] == 'training'
small = tracks['set', 'subset'] <= 'medium'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

y_train = tracks.loc[medium_data_train_sort.track_id, ('track', 'genre_top')]
y_test = tracks.loc[medium_data_train_sort.track_id, ('track', 'genre_top')]
X_train = features.loc[medium_data_train_sort.track_id, 'mfcc']
X_test = features.loc[medium_data_train_sort.track_id, 'mfcc']

print('{} training examples, {} testing examples'.format(y_train.size, y_test.size))
print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

19922 training examples, 19922 testing examples
140 features, 16 classes


# Train

In [16]:
from kddirkit.networks.encoders import SentenceEncoder
from kddirkit.networks.models import BaselineModel
from kddirkit.config import *
from kddirkit.dataloaders import LoadFMA, LoadHierData
from kddirkit.frameworks import Trainer
from kddirkit.losses.FocalLoss import FocalLoss

parser = Parser(ROOT_PATH + "/data/config", "HMAN")
oneParser = parser.oneParser
args, _ = oneParser.parse_known_args(args=[])
args = vars(args)

In [17]:
use_cuda = not args['no_cuda'] and torch.cuda.is_available()

torch.manual_seed(args['seed'])

device = torch.device("cuda" if use_cuda else "cpu")

HierDataLoader = LoadHierData.HierDataLoader(workdir=os.getcwd(), pattern='default', device=device)
genre_levels_Tensor = HierDataLoader.genre_levels_Tensor.to(device)
genre_level_layer = HierDataLoader.genre_level_layer

trainDataLoader = LoadFMA.FMATrainDataLoader(device=device)
testDataLoader = LoadFMA.FMATestDataLoader(mode="pr", device=device)

In [18]:
col_name = ['track_id', 'album_id', 'album_type', 'artist_id', 'set_split', 'set_subset', 'track_genres_top', 'track_genre', 'track_genres_all']
medium_data_train_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_train_sort.txt', sep ='-----',  skiprows =1, names  = col_name)
medium_data_val_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_VAL_sort.txt', sep = '-----',  skiprows =1, names  = col_name)
medium_data_test_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_test_sort.txt', sep = '-----', skiprows =1, names  = col_name)

C:\Users\Johan\AppData\Local\Temp\ipykernel_38168\691608824.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  medium_data_train_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_train_sort.txt', sep ='-----',  skiprows =1, names  = col_name)
C:\Users\Johan\AppData\Local\Temp\ipykernel_38168\691608824.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  medium_data_val_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_VAL_sort.txt', sep = '-----',  skiprows =1, names  = col_name)
C:\Users\Johan\AppData\Local\Temp\ipykernel_38168\691608824.py:4: ParserWarning: Falling back to the 'python' engine 

In [ ]:
"""@nni.variable(nni.choice(50, 100, 160), name=training_batch_size)"""
training_batch_size = 160

"""@nni.variable(nni.choice(0.2, 0.3, 0.1), name=learning_rate)"""
learning_rate = 0.2

weight_decay = args['weight_decay']

args['num_classes'] = 16
keep_prob = args['keep_prob']

hidden_size = 140

In [ ]:
sentence_encoder = SentenceEncoder.MusicCnnEncoder(trainDataLoader.feature[0].shape[0], trainDataLoader.feature[0].shape[1], hidden_size)

AttributeError: 'FMATrainDataLoader' object has no attribute 'feature'

In [ ]:
model = BaselineModel.AttentionNetwork16(sentence_encoder = sentence_encoder,
                                        genre_levels=genre_levels_Tensor,
                                        genre_level_layer=genre_level_layer,
                                        keep_prob=keep_prob,
                                        train_batch_size=training_batch_size,
                                        test_batch_size=args['testing_batch_size'],
                                        num_classes=args['num_classes'],
                                        device=device).to(device)

In [ ]:
parameters_to_optimize = filter(lambda x: x.requires_grad, model.parameters())
optimizer = optim.SGD(parameters_to_optimize,
                      learning_rate,
                      weight_decay=weight_decay)

trainer = Trainer.Trainer(model=model,
                          FMATrainDataLoader=trainDataLoader,
                          FMATestDataLoader=testDataLoader,
                          args=args,
                          epoch=0,
                          training_batch_size=training_batch_size,
                          criterion=criterion,
                          optimizer=optimizer, learning_rate=learning_rate,
                          device=device, useScheduler=False, warmup=False)

In [ ]:
"""@nni.variable(nni.choice("pcnn"), name=encoderName)"""
encoderName = "cnn"

if encoderName == "cnn":
    sentence_encoder = SentenceEncoder.CNNSentenceEncoder(SkipGramVec, 120, args['pos_num'], word_size, pos_size,
                                                          hidden_size)
elif encoderName == "pcnn":
    sentence_encoder = SentenceEncoder.PCNNSentenceEncoder(SkipGramVec, 120, args['pos_num'], word_size, pos_size,
                                                           hidden_size)

# """@nni.variable(nni.choice('HAttentionNetwork', 'MKGV2AttentionNetworkAg', 'MKV2AttentionNetworkAg', 'MKV2AttentionNetworkWAGT2', 'MKV2AttentionNetworkWAGK2', 'MKMirrorDotAttentionNetworkWAG', 'MKDoubleDotAttentionNetworkWAG'), name=modelName)"""
"""@nni.variable(nni.choice('MKV2AttentionNetworkAg') , name=modelName)"""
modelName = "MKMirrorDotAttentionNetworkWAG"
